In [19]:
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt

In [20]:
def load_data(fname):
    with open(fname, "rb") as f:
        return pkl.load(f)

In [21]:
def get_val_accs(data, iterations=15):
    val_accs = []
    for seed_data in data:

        iteration_val_accs = []
        for iteration_data in seed_data:
            iteration_val_acc = iteration_data['train']["test_acc"]
            iteration_val_accs.append(iteration_val_acc)
        val_accs.append(iteration_val_accs)
    return np.array(val_accs)


def get_neurons(data, iterations=15):
    neurons = []
    for seed_data in data:

        iteration_neurons_l0 = []
        iteration_neurons_l1 = []
        for iteration_data in seed_data:
            iteration_neuron = iteration_data['prune'][0]["neurons"]
            iteration_neurons_l0.append(iteration_neuron)
            iteration_neuron = iteration_data['prune'][1]["neurons"]
            iteration_neurons_l1.append(iteration_neuron)
        neurons.append((iteration_neurons_l0, iteration_neurons_l1))
    return neurons


def get_ckas(data, iterations=15):
    ckas = []
    for seed_data in data:

        iteration_ckas = []
        for iteration_data in seed_data:
            iteration_cka = min(x["ckas"][-1] for x in iteration_data['prune'])
            iteration_ckas.append(iteration_cka)
        ckas.append(iteration_ckas)
    return np.array(ckas)

In [22]:
cka_data = load_data(f"_in-784_hd-512-512_out-10_dr-0.5_strategy-iterative_type-cka.pkl")
cka_val_accs = get_val_accs(cka_data)
cka_ckas = get_ckas(cka_data)

l1_data = load_data(f"_in-784_hd-512-512_out-10_dr-0.5_strategy-iterative_type-l1.pkl")
l1_val_accs = get_val_accs(l1_data)
l1_ckas = get_ckas(l1_data)

In [23]:
cka_neurons = get_neurons(cka_data)
l1_neurons = get_neurons(l1_data)


print(len(cka_neurons[0]))

l0_all =[]
l1_all = []
for seed_cka_neurons, seed_l1_neurons in zip(cka_neurons, l1_neurons):
    layer0 = []
    layer1 = []

    ckap, l1p = [], []
    for cka_n, l1_n in zip(seed_cka_neurons[0], seed_l1_neurons[0]): 
        # print(cka_n)
        # ckap.extend(cka_n)
        # l1p.extend(l1_n)
        ckap, l1p = cka_n, l1_n
        layer0.append(len(set(ckap).intersection(l1p)))
    
    ckap, l1p = [], []
    for cka_n, l1_n in zip(seed_cka_neurons[1], seed_l1_neurons[1]): 
        # ckap.extend(cka_n)
        # l1p.extend(l1_n)
        ckap, l1p = cka_n, l1_n
        layer1.append(len(set(ckap).intersection(l1p)))

    l0_all.append(layer0)
    l1_all.append(layer1)


l0_all = np.array(l0_all)
l1_all = np.array(l1_all)
print(l0_all)

l0_all += l1_all


l0_all = np.array(l0_all, dtype=float)

pruned_prop = 0
for i in range(15):
    pruned_prop = (0.2) * (0.8 ** (i - 1))
    l0_all[:, i] /= 2 * pruned_prop * 512

print(l0_all)

for i in range(15):
    print(i, l0_all[:, i].mean(), l0_all[:, i].max(), l0_all[:, i].min())

# print(l0_all)
# print(l1_all)

2
[[55 23  7  3  6  4  3  1  1  0  1  0  1  0  0]
 [56 17 11  3  5  4  1  1  0  1  0  2  0  0  0]
 [55 17 13  7  3  6  1  3  0  0  0  0  0  0  0]
 [49 15 10  3  3  3  1  0  1  1  1  0  0  0  0]
 [62 19 12  9  6  2  1  2  0  0  2  1  1  1  1]
 [56 18  8  5  5  3  1  3  0  0  1  0  0  0  1]
 [64 13 12  5  4  3  2  0  1  0  1  0  0  0  0]
 [51 19  6 10  4  4  1  0  2  1  0  1  1  0  0]
 [56 16 11  6  5  3  1  1  2  0  1  1  0  0  0]]
[[0.421875   0.21972656 0.1159668  0.09155273 0.08583069 0.09536743
  0.04470348 0.01862645 0.04656613 0.08731149 0.07275958 0.04547474
  0.05684342 0.         0.        ]
 [0.41796875 0.18066406 0.15258789 0.09155273 0.09536743 0.07152557
  0.04470348 0.05587935 0.04656613 0.02910383 0.         0.13642421
  0.05684342 0.         0.        ]
 [0.4375     0.18554688 0.16479492 0.12207031 0.04768372 0.13113022
  0.02980232 0.09313226 0.04656613 0.         0.         0.
  0.         0.         0.        ]
 [0.4375     0.18554688 0.14648437 0.08392334 0.08583069 

In [28]:
s = """0 0.4405381944444444 0.48046875 0.4140625
1 0.1953125 0.2197265625 0.1708984375
2 0.13631184895833331 0.16479492187499997 0.09155273437499999
3 0.10511610243055552 0.14495849609374997 0.07629394531249999
4 0.09112887912326387 0.15258789062499994 0.047683715820312486
5 0.07682376437717012 0.13113021850585935 0.047683715820312486
6 0.04139211442735459 0.07450580596923825 0.014901161193847651
7 0.05174014303419322 0.09313225746154781 0.01862645149230956
8 0.04656612873077391 0.06984919309616086 0.023283064365386956
9 0.04527262515491907 0.08731149137020107 0.0
10 0.04850638409455615 0.14551915228366843 0.0
11 0.06063298011819518 0.13642420526593915 0.0
12 0.025263741715914657 0.05684341886080798 0.0
13 0.00789491928622333 0.07105427357600998 0.0
14 0.01973729821555832 0.08881784197001245 0.0"""
lines = s.split("\n")
for line in lines:
    d = line.split(" ")
    print(d[0], d[1], float(d[2]) - float(d[1]), float(d[1]) - float(d[3]))

0 0.4405381944444444 0.03993055555555558 0.02647569444444442
1 0.1953125 0.0244140625 0.0244140625
2 0.13631184895833331 0.028483072916666657 0.04475911458333333
3 0.10511610243055552 0.03984239366319445 0.02882215711805554
4 0.09112887912326387 0.06145901150173608 0.04344516330295138
5 0.07682376437717012 0.05430645412868923 0.02914004855685763
6 0.04139211442735459 0.033113691541883664 0.02649095323350694
7 0.05174014303419322 0.04139211442735459 0.033113691541883664
8 0.04656612873077391 0.02328306436538695 0.023283064365386956
9 0.04527262515491907 0.042038866215282 0.04527262515491907
10 0.04850638409455615 0.09701276818911228 0.04850638409455615
11 0.06063298011819518 0.07579122514774397 0.06063298011819518
12 0.025263741715914657 0.03157967714489332 0.025263741715914657
13 0.00789491928622333 0.06315935428978664 0.00789491928622333
14 0.01973729821555832 0.06908054375445413 0.01973729821555832


In [25]:
# mean_cka_acc = cka_val_accs.mean(axis=0)
# mean_l1_acc = l1_val_accs.mean(axis=0)

# max_cka_acc = np.abs(mean_cka_acc - cka_val_accs.max(axis=0))
# max_l1_acc = np.abs(mean_l1_acc - l1_val_accs.max(axis=0))

# min_cka_acc = np.abs(mean_cka_acc - cka_val_accs.min(axis=0))
# min_l1_acc = np.abs(mean_l1_acc - l1_val_accs.min(axis=0))

# x = [0.2 * (0.8) ** (i - 1) for i in range(1, mean_cka_acc.shape[0] + 1)]
# for i in range(1, len(x)):
#     x[i] += x[i-1]
# # x = [np.round(v, 2) for v in x]
# x = [1 - v for v in x]

# print(x[::2])
# print()

# for i in range(mean_cka_acc.shape[0]):
#     print(int(np.round(x[i] * size)), mean_cka_acc[i], max_cka_acc[i], min_cka_acc[i])
# print()

# for i in range(mean_cka_acc.shape[0]):
#     print(int(np.round(x[i] * size)), mean_l1_acc[i], max_l1_acc[i], min_l1_acc[i])

# # std_cka_acc = cka_val_accs.std(axis=0)
# # std_l1_acc = l1_val_accs.std(axis=0)

# # print(len(cka_data[0][0]['prune'][0]['neurons']))

# # mean_cka_cka = cka_ckas.mean(axis=0)
# # mean_l1_cka = l1_ckas.mean(axis=0)

In [26]:
mean_cka_acc = cka_val_accs.mean(axis=0)
mean_l1_acc = l1_val_accs.mean(axis=0)

max_cka_acc = np.abs(mean_cka_acc - cka_val_accs.max(axis=0))
max_l1_acc = np.abs(mean_l1_acc - l1_val_accs.max(axis=0))

min_cka_acc = np.abs(mean_cka_acc - cka_val_accs.min(axis=0))
min_l1_acc = np.abs(mean_l1_acc - l1_val_accs.min(axis=0))

x = [0.1 * (0.9) ** (i - 1) for i in range(1, mean_cka_acc.shape[0] + 1)]
for i in range(1, len(x)):
    x[i] += x[i-1]
x = [np.round(v, 2) for v in x]
print(x)
x = x[::4]

print(x)
print()

for i in range(mean_cka_acc.shape[0]):
    print(i, mean_cka_acc[i], max_cka_acc[i], min_cka_acc[i])
print()

for i in range(mean_cka_acc.shape[0]):
    print(i, mean_l1_acc[i], max_l1_acc[i], min_l1_acc[i])

# std_cka_acc = cka_val_accs.std(axis=0)
# std_l1_acc = l1_val_accs.std(axis=0)

# print(len(cka_data[0][0]['prune'][0]['neurons']))

# mean_cka_cka = cka_ckas.mean(axis=0)
# mean_l1_cka = l1_ckas.mean(axis=0)

[0.1, 0.19, 0.27, 0.34, 0.41, 0.47, 0.52, 0.57, 0.61, 0.65, 0.69, 0.72, 0.75, 0.77, 0.79]
[0.1, 0.41, 0.61, 0.75]

0 0.9832111111111109 0.001388888888889106 0.0011111111111109517
1 0.9825555555555555 0.0014444444444444704 0.0018555555555554992
2 0.9817222222222222 0.0005777777777777882 0.00042222222222221273
3 0.9808777777777777 0.0007222222222222907 0.001277777777777711
4 0.9791777777777777 0.0012222222222223467 0.001677777777777667
5 0.9772000000000001 0.0007999999999999119 0.0007000000000000339
6 0.9753555555555555 0.0017444444444444374 0.0012555555555555653
7 0.9723555555555554 0.0012444444444446034 0.0014555555555554323
8 0.9686666666666667 0.0009333333333333416 0.0009666666666666712
9 0.9631777777777777 0.0020222222222222586 0.002677777777777668
10 0.9566888888888889 0.0022111111111110526 0.002888888888888941
11 0.9489000000000001 0.0032999999999999696 0.0029000000000001247
12 0.9387666666666666 0.0029333333333333433 0.003566666666666607
13 0.9270333333333333 0.00446666666666673 

In [27]:

# x = list(range(len(mean_l1_val_acc)))
# yerr = [
#     np.abs(mean_l1_val_acc - l1_val_accs.min(axis=0)), 
#     np.abs(mean_l1_val_acc - l1_val_accs.max(axis=0)),
# ]
# plt.errorbar(x, mean_l1_val_acc, yerr=yerr, label='l1')
# yerr = [
#     np.abs(mean_cka_val_acc - cka_val_accs.min(axis=0)), 
#     np.abs(mean_cka_val_acc - cka_val_accs.max(axis=0)),
# ]
# plt.errorbar(x, mean_cka_val_acc, yerr=yerr, label='cka')

# plt.title("Iterative pruning with L1 and CKA")
# plt.xlabel("Proportion of neurons in fc0 and fc1 pruned")
# plt.ylabel("Validation accuracy")

# plt.legend()
# plt.show()

# plt.plot(x, mean_l1_cka, label='l1_cka')
# plt.plot(x, mean_cka_cka, label='cka_cka')
# plt.legend()
# plt.show()